In [8]:
## Spell checker from Peter Norvig. Not perfect, but works.
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [9]:
## Import all modules needed
## lem is lemmatizing the words
from nltk.corpus import wordnet
def lem(x):
    sentence = ''
    for x in re.sub('[^a-z0-9 ]','',x).split():   
        sentence = sentence + ' ' + lemmatizer.lemmatize(correct(x))
    count = 0
    return sentence

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
%matplotlib inline
from nltk import word_tokenize
class LemmaTokenizer(object):
     def __init__(self):
         self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(min_df=10,stop_words=stops,max_features=1000,ngram_range=(1,3),tokenizer=LemmaTokenizer())
tfvectorizer = TfidfVectorizer(stop_words=stops,min_df=10,max_features=1000,ngram_range=(1,3),tokenizer=LemmaTokenizer())

### Take note of the assumptions made in the vectorizer specifications. There are two kinds of vectorizers initialized: count and tfidf. I've limited both to consider only n-grams that appear at least ten times. I've also limited the feature set into the top 1,000 n-grams that appear the most often in the reviews. Also it only extracts unigrams to trigrams. You can edit any of the parameters.

### You can switch between count and tfidf vectorizers by changing between "vectorizer" and "tfvectorizer" in one of the cells below

In [10]:
import pandas as pd
import pickle
with open("C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/review_business_df.pkl", 'rb') as picklefile: 
    review_business_df = pickle.load(picklefile)

In [11]:
sample_df = review_business_df.sample(10000,random_state=1).dropna().reset_index(drop=True)

### Note that I've only extract 10,000 sample reviews. This is just to manage its tracktability. Lemmatizing takes a really long time. Indirectly, 10,000 reviews is also big enough relative to the 1,000 features (discussed above).

In [12]:
short_df = sample_df.iloc[:,0:2]

In [7]:
short_df['spell'] = short_df['text'].apply(lambda x: x.lower())

In [8]:
short_df['s_spell_lem'] = short_df['spell'].apply(lem)

In [101]:
new_df = short_df[['s_spell_lem','stars_x']]

In [102]:
corp = list(new_df.s_spell_lem)

### You can change the vectorizer type in the cell below

In [122]:
feature = tfvectorizer.fit_transform(corp)

In [139]:
## Turns count/tfidf matrix into a dataframe
featuredf = pd.DataFrame(feature.A, columns=tfvectorizer.get_feature_names())

In [140]:
## Concat Y and X
new_df = pd.concat((new_df,featuredf),axis=1)

In [141]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 3002 entries, s_spell_lem to young
dtypes: float64(3000), object(2)
memory usage: 229.0+ MB


In [14]:
def string(x):
    if x == 5:
        return "five"
    elif x == 4:
        return 'four'
    elif x == 3:
        return 'three'
    elif x == 2:
        return 'two'
    elif x == 1:
        return 'one'

In [15]:
new_df['stars_x'] = short_df['stars_x'].apply(string)

In [16]:
new_df.head()

,s_spell_lem,stars_x,100,able,absolutely,ache,across,actually,add,added,...,would definitely,would recommend,wrap,wrong,year,yelp,yes,yet,york,young
0,i go out to eat often and have been to many p...,four,0.0,0.0,0.0,0.0,0.126137,0.103447,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,always happysatisfied when i leave they have ...,five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,i dont like fried chicken there i said it sou...,five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,love this little hidden gem best go by far in...,five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a bit conflict about this cafe because last n...,three,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.062492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pickle
with open('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/kenn_review_df.pkl', 'wb') as picklefile: 
    pickle.dump(new_df, picklefile)

In [1]:
import pickle
import pandas as pd
import numpy as np
with open("C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/kenn_review_df.pkl", 'rb') as picklefile: 
    new_df = pickle.load(picklefile)

In [2]:
collapse_df = new_df.copy()

In [3]:
##Apply this if you want to collapse the 5-star ratings by removing 3-star reviews and combining 1 & 2 stars 
##and 4 & 5 ratings

def collapse(x):
    if x == 'four' or x == 'five':
        return 'four/five'
    elif x == 'one' or x == 'two':
        return 'one/two'
    elif x == 'three':
        return 'three'

collapse_df.stars_x = new_df.stars_x.apply(collapse)

In [4]:
# Run if you want to delete 3-stars
# collapse_df = collapse_df[new_df['stars_x'] != 'three']

In [5]:
collapse_df.head()

,s_spell_lem,stars_x,100,able,absolutely,ache,across,actually,add,added,...,would definitely,would recommend,wrap,wrong,year,yelp,yes,yet,york,young
0,i go out to eat often and have been to many p...,four/five,0.0,0.0,0.0,0.0,0.126137,0.103447,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,always happysatisfied when i leave they have ...,four/five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,i dont like fried chicken there i said it sou...,four/five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,love this little hidden gem best go by far in...,four/five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a bit conflict about this cafe because last n...,three,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.062492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
##Adds sentiment as a feature. Note that I added 1 because some algorithms won't accept negative sentiment scores
##Sentiment scores is based on TextBlob where it goes from -1.0 to 1.0 (negative to positive)
##Change the reference DF if you want to go back to using the 5-star categories

from textblob import TextBlob

collapse_df['senti'] = collapse_df['s_spell_lem'].apply(lambda x: TextBlob(x).sentiment[0] + 1)

collapse_df.head()

,s_spell_lem,stars_x,100,able,absolutely,ache,across,actually,add,added,...,would recommend,wrap,wrong,year,yelp,yes,yet,york,young,senti
0,i go out to eat often and have been to many p...,four/five,0.0,0.0,0.0,0.0,0.126137,0.103447,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.234848
1,always happysatisfied when i leave they have ...,four/five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.578571
2,i dont like fried chicken there i said it sou...,four/five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.182030
3,love this little hidden gem best go by far in...,four/five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.249167
4,a bit conflict about this cafe because last n...,three,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.062492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.279425


# Model testing

In [7]:
##Split into train and test at 75/25
##Change the reference DF if you want to go back to using the 5-star categories
from sklearn.cross_validation import train_test_split 
train, test = train_test_split(collapse_df,test_size = 0.25)

In [8]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [17]:
## KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5).fit(X_train,Y_train)
knn_Y_pred = knn.predict(X_test)
print("KNN Accuracy: ",np.mean(knn_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,knn_Y_pred,labels=['one/two','three','four/five']))
print(classification_report(Y_test,knn_Y_pred))

KNN Accuracy:  0.5644
[[ 162  213  107]
 [  37  204  132]
 [  66  534 1045]]
             precision    recall  f1-score   support

  four/five       0.81      0.64      0.71      1645
    one/two       0.61      0.34      0.43       482
      three       0.21      0.55      0.31       373

avg / total       0.69      0.56      0.60      2500



In [10]:
## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,nb_Y_pred,labels=['one/two','three','four/five']))
print(classification_report(Y_test,nb_Y_pred))

NB Accuracy:  0.7848
[[ 345   50   87]
 [  80  105  188]
 [  64   69 1512]]
             precision    recall  f1-score   support

  four/five       0.85      0.92      0.88      1645
    one/two       0.71      0.72      0.71       482
      three       0.47      0.28      0.35       373

avg / total       0.76      0.78      0.77      2500



In [11]:
## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression().fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,log_Y_pred,labels=['one/two','three','four/five']))
print(classification_report(Y_test,log_Y_pred))

Logistc Accuracy:  0.792
[[ 350   40   92]
 [  83   75  215]
 [  43   47 1555]]
             precision    recall  f1-score   support

  four/five       0.84      0.95      0.89      1645
    one/two       0.74      0.73      0.73       482
      three       0.46      0.20      0.28       373

avg / total       0.76      0.79      0.77      2500



In [14]:
## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcl_Y_pred,labels=['one/two','three','four/five']))
print(classification_report(Y_test,svcl_Y_pred))

SVC Linear Accuracy:  0.7824
[[ 348   55   79]
 [  87   86  200]
 [  53   70 1522]]
             precision    recall  f1-score   support

  four/five       0.85      0.93      0.88      1645
    one/two       0.71      0.72      0.72       482
      three       0.41      0.23      0.29       373

avg / total       0.75      0.78      0.76      2500



In [15]:
## RBF SVC
from sklearn.svm import SVC
svcrbf = SVC(kernel='rbf', gamma=1).fit(X_train,Y_train)
svcrbf_Y_pred = svcrbf.predict(X_test)
print("SVC RBF Accuracy: ",np.mean(svcrbf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcrbf_Y_pred,labels=['one/two','three','four/five']))
print(classification_report(Y_test,svcrbf_Y_pred))

SVC RBF Accuracy:  0.7404
[[ 212    2  268]
 [  24    0  349]
 [   6    0 1639]]
             precision    recall  f1-score   support

  four/five       0.73      1.00      0.84      1645
    one/two       0.88      0.44      0.59       482
      three       0.00      0.00      0.00       373

avg / total       0.65      0.74      0.67      2500



In [12]:
## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,dt_Y_pred,labels=['one/two','three','four/five']))
print(classification_report(Y_test,dt_Y_pred))

Decision Tree Accuracy:  0.6716
[[ 241  108  133]
 [  81   99  193]
 [ 108  198 1339]]
             precision    recall  f1-score   support

  four/five       0.80      0.81      0.81      1645
    one/two       0.56      0.50      0.53       482
      three       0.24      0.27      0.25       373

avg / total       0.67      0.67      0.67      2500



In [13]:
## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,rf_Y_pred,labels=['one/two','three','four/five']))
print(classification_report(Y_test,rf_Y_pred))

Random Forests Accuracy:  0.7544
[[ 279   39  164]
 [  73   44  256]
 [  43   39 1563]]
             precision    recall  f1-score   support

  four/five       0.79      0.95      0.86      1645
    one/two       0.71      0.58      0.64       482
      three       0.36      0.12      0.18       373

avg / total       0.71      0.75      0.72      2500



In [51]:
## Modified off-the-shelf sentiment analyzer
import nltk.sentiment.util
def demo_liu_hu_lexicon2(sentence, plot=False):
    print('k')
    """
    Basic example of sentiment classification using Liu and Hu opinion lexicon.
    This function simply counts the number of positive, negative and neutral words
    in the sentence and classifies it depending on which polarity is more represented.
    Words that do not appear in the lexicon are considered as neutral.

    :param sentence: a sentence whose polarity has to be classified.
    :param plot: if True, plot a visual representation of the sentence polarity.
    """
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank

    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]

    x = list(range(len(tokenized_sent))) # x axis for the plot
    y = []

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
            y.append(1) # positive
        elif word in opinion_lexicon.negative():
            neg_words += 1
            y.append(-1) # negative
        else:
            y.append(0) # neutral

    return (pos_words - neg_words)
#     if pos_words > neg_words:
#         return 'Positive'
#     elif pos_words < neg_words:
#         return 'Negative'
#     elif pos_words == neg_words:
#         return 'Neutral'

    if plot == True:
        _show_plot(x, y, x_labels=tokenized_sent, y_labels=['Negative', 'Neutral', 'Positive'])